In [ ]:
import os
import glob
from osgeo import gdal

In [2]:


def create_vrt_and_convert_to_cog(input_folder, output_folder):
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Path for VRT file
    vrt_path = os.path.join(output_folder, "merged.vrt")
    
    # Path for final COG file
    output_cog_path = os.path.join(output_folder, "final_output_cog.tif")

    # Find all .tif files in input folder and subfolders
    input_files = glob.glob(os.path.join(input_folder, "**", "*.tif"), recursive=True)

    # Create VRT
    vrt_options = gdal.BuildVRTOptions(resampleAlg='nearest', addAlpha=False)
    vrt = gdal.BuildVRT(vrt_path, input_files, options=vrt_options)
    vrt = None  # Close the dataset

    # Convert VRT to COG
    gdal.SetConfigOption('GDAL_CACHEMAX', '1024')  # Increase cache size to 1 GB
    gdal.SetConfigOption('GDAL_NUM_THREADS', 'ALL_CPUS')  # Use all available CPUs
    
    translate_options = gdal.TranslateOptions(
        format='COG',
        creationOptions=[
            "COMPRESS=LZW",
            "PREDICTOR=2",
            "BIGTIFF=YES",
            "TILED=YES",
            "BLOCKSIZE=512",
            "OVERVIEW_RESAMPLING=AVERAGE",
            "NUM_THREADS=ALL_CPUS"
        ]
    )
    
    gdal.Translate(output_cog_path, vrt_path, options=translate_options)

    # Remove temporary VRT file
    os.remove(vrt_path)

# Example usage
input_folder = '/Volumes/WD Green/Data/DTM/Sudbury/Sudbury-DTM-11/DTM'
output_folder = '/Volumes/WD Green/Data/DTM/Sudbury/Sudbury-DTM-11'
create_vrt_and_convert_to_cog(input_folder, output_folder)

Warning 6: driver COG does not support creation option TILED


In [ ]:
def upload_to_aws(local_file, bucket, s3_folder):
    s3 = boto3.client('s3')
    
    try:
        file_name = os.path.basename(local_file)
        s3_file = os.path.join(s3_folder, file_name)
        
        print(f"Uploading {file_name} to AWS S3...")
        s3.upload_file(local_file, bucket, s3_file)
        print(f"Upload Successful. File available at: https://{bucket}.s3.amazonaws.com/{s3_file}")
        return True
    except FileNotFoundError:
        print("The file was not found")
        return False
    except NoCredentialsError:
        print("Credentials not available")
        return False

In [ ]:
# Upload to S3
if cog_file:
    upload_to_aws(cog_file, 'ccemp-bucket', 'Public Access/COG/')
else:
    print("COG creation failed, no upload attempted.")